In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
sns.set()
from geopy.geocoders import Nominatim

import pymysql
import sqlalchemy as db

PREPROCESAMIENTO

Carga de dataset de accidentes (AccidentesAviones.csv)

In [2]:
df_oaci = pd.read_csv('AccidentesAviones.csv', quotechar='"', na_values='?') # lectura de csv con datos de accidentes
df_oaci.drop(columns='Unnamed: 0', inplace=True) # eliminació de columna con indice preestablecido.

Transformación de Fecha y hora a formato datetime

In [3]:
df_oaci['HORA declarada'] = df_oaci['HORA declarada'].map(lambda x: x.replace('c',"").strip().replace(":","") if type(x)==str and ('c' in x or ':' in x) else x ) # unficación de columna 'HORA declarada' para que todos queden con formato %H%M
df_oaci['HORA declarada'] = df_oaci['HORA declarada'].str.strip() #Eliminación de espacios al comienzo y al final de la cadena

In [4]:
df_oaci.fecha= pd.to_datetime(df_oaci.fecha, format='%B %d, %Y') #definición y cambio de formato de columna fecha a mes-dia-año

In [5]:
df_oaci['HORA declarada'] = pd.to_datetime(df_oaci['HORA declarada'] , format='%H%M', errors='coerce').dt.time # cambio de formato de columna 'HORA declarada' a hora minutos

Reasignación de nombres de columna

In [7]:
# cambio de nombres de columnas
df_oaci.rename(columns={
            'fecha':'date',
            'HORA declarada':'hour',
            'Ruta':'place',
            'OperadOR':'operator',
            'PASAJEROS A BORDO':'pass_aboard',
            'cantidad de fallecidos':'total_fatalities',
            'passenger_fatalities':'pass_fatalities'} , inplace=True)

Conversión a mayúsculas de columnas de texto

In [8]:
#converción a mayúsculas de columnas de texto
df_oaci.place= df_oaci.place.map(lambda x: x.upper() if type(x)==str else x)
df_oaci.operator= df_oaci.operator.map(lambda x: x.upper() if type(x)==str else x)
df_oaci.route= df_oaci.route.map(lambda x: x.upper() if type(x)==str else x)
df_oaci.ac_type= df_oaci.ac_type.map(lambda x: x.upper() if type(x)==str else x)
df_oaci.summary= df_oaci.summary.map(lambda x: x.upper() if type(x)==str else x)

Eliminación de columnas que no ofrecen información

In [9]:
# eliminación de columnas inútiles
df_oaci.drop(columns=['flight_no', 'cn_ln'], inplace=True)

Unificación de categorías escritas de varios modos

In [10]:
# corrección de palabras con igual significado pero escritas de manera distinta
df_oaci.operator = df_oaci.operator.map(lambda x: 'MILITARY - ROYAL AIR FORCE' if type(x)==str and x=='MILITARY -ROYAL AIR FORCE' else x) #unifica MILITARY - ROYAL AIR FORCE y MILITARY -ROYAL AIR FORCE
lista_usairforce = ['MILITARY - U.S. ARMY AIR FORCES', 'MILITARY - U.S. AIR FORCE / U.S. AIR FORCE' , 'MILITARY - U.S. AIR FORCE / MILITARY - U.S. AIR FORCE', 'MILTIARY - U.S. AIRFORCE', 'MILITARY - U.S. ARMY AIR FORCE', 'MILITARY - U.S. AIR FORCE / MILITARY -  U.S. AIR FORCE', 'MILITARY - U.S. AIR FORCE/MILITARY - U.S. AIR FORCE', 'MILITARY - U.S. ARMY AIR FORCES / MILITARY - U.S. ARMY AIR FORCES', 'MILITARY -  (2) U.S. ARMY AIR FORCES']
df_oaci.operator = df_oaci.operator.map(lambda x: 'MILITARY - U.S. AIR FORCE' if type(x)==str and x in lista_usairforce  else x) # unifica U.S. arif force
df_oaci.route = df_oaci.route.map(lambda x: 'TEST' if type(x)==str and (x=='TEST FLIGHT' or x=='TESTING')  else x)

Correción de fila con 'pass_aboard' > 'all_aboard'

In [42]:
for i in df_oaci[df_oaci['pass_aboard'] > df_oaci['all_aboard']].index.to_list() :
    df_oaci.all_aboard.loc[i] = df_oaci.pass_aboard[i] + df_oaci.crew_aboard[i]

C:\Users\jcgut\AppData\Local\Temp\ipykernel_15400\3795857078.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_oaci.all_aboard.loc[i] = df_oaci.pass_aboard[i] + df_oaci.crew_aboard[i]


Obtención del pais del lugar del accidente(columna 'place')

In [21]:
geolocator = Nominatim(user_agent='aero') # instancia geocolalizador de geopy

In [22]:
# definición de función que devuelve el pais de un lugar de la columna 'place' utilizando el geolocalizador de geopy
def pais(place): 
    if type(place)==str:
        noloc=['NEAR' , 'OFF' ,'OVER', 'NORTH SEA','NORTHERN', 'WESTERN' ]
        for i in noloc:                 #elimina las palabras de la lista noloc del lugar del accidente para que puedan ser reconocidos por el geolocalizador
            if i in place:
                    place = place.replace(i,"").strip()
                    continue
# se realiza manejo de excepciones para saltar detenciones por falta de reconocimiento del lugar               
        try:
            return geolocator.geocode(place, language='en').address.split(',')[-1].strip()
        except TimeoutError:    
            return None
        except AttributeError:
            return None
            #print (place, type(place))
    else: 
        return place

In [31]:
# devuelve el pais basado en el lugar del accidente (columna 'place') utilizando la funcion definida en la celda anterior.
# Este proceso tarda alrededor de 45 min por lo que el resultado se exportó al archivo col_country.csv y puede ser cargado en 
# la siguiente celda para saltarse este paso.

#col_pais=[]
#for i in df_oaci.place:
#    col_pais.append(pais(i))

In [26]:
# ejecutar para cargar la columna paises si no se desea ejecutar el geolocalizador de geopy de la celda anterior
col_pais = pd.read_csv('col_country.csv')

In [28]:
df_oaci.insert(3, 'country', col_pais.country) # Inserción de columna de paises en el dataframe
df_oaci.country= df_oaci.country.map(lambda x: x.upper() if type(x)==str else x) # conversión de datos de columna country a mayúsculas

In [37]:
# exportación de columna de paises obtenida del geolocalizador de geopy, esto se hace debido a que ejecutar el geolocalizador 
# sobre toda la columna 'place', se tarda alrededor de 45 min y algunas veces pierde la conexion obligando a repetir el proceso
#df_oaci.country.to_csv('col_country.csv', index=False) 

Carga de dataset de pasajeros transportados por via aérea por año y pais, descargado del databank del banco mundial(cantidad_pasajeros.csv) 
(https://datos.bancomundial.org/indicador/IS.AIR.PSGR?end=2020&name_desc=false&start=1970&view=chart&year=1974)

In [129]:
df_pass = pd.read_csv('cantidad_pasajeros.csv', quotechar='"') # lectura de csv con datos de pasajeros por año y pais
df_pass.drop(columns=['Indicator Name', 'Indicator Code', 'Country Code',
'1960', '1961', '1962', '1963', '1964', '1965', '1966', '1967', '1968','1969','2021'], inplace=True) # eliminación de filas inútiles o sin datos

Reorganización de la tabla

In [130]:
# reorganización de la tabla para obtener solo tres columnas : country , anio, passengers, sin valores nulos.
df_pass = df_pass.melt('Country Name', var_name='anio', value_name='passengers').dropna(axis=0, how='any').reset_index(drop=True)

Cambios de nombre y formato de columnas 

In [131]:
df_pass.rename(columns={'Country Name':'country'} , inplace=True) # renombrar columna 'Country Name'
df_pass['country'] = df_pass['country'].map(lambda x: x.upper() if type(x)==str else x) # converción a mayúsculas datos de columna country
df_pass = df_pass.astype({'passengers':np.int64 , 'anio':np.int64}) # converción a entero de columnas passengers y anio
df_pass.anio = pd.to_datetime(df_pass.anio, format='%Y') #Asignación de formato tipo fecha de columna anio.

CARGA DE DATASET A SQL

In [143]:
#establecimiento de conexión con la base de datos 'oaci' del localhost
database_username='root' # Nombre de cliente en MySQL Workbench
database_password='4488' # Contraseña de servidor MySQL Workbench
database_ip='localhost' # localizacion del servidor
database_name='oaci' # Nombre de Base de datos a la que nos conectaremos
engine=db.create_engine(f'mysql+pymysql://{database_username}:{database_password}@{database_ip}/{database_name}')
conexion=engine.connect()

In [58]:
# definicion previa de un diccionario con los tipos con los que se desea ingestar la data en la base de datos 
# y envío de datos de accidentes a la base de datos

types_oaci={'date':db.types.DATE,                               
 'hour':db.types.TIME,
 'place':db.types.NVARCHAR(length=80),
 'country':db.types.NVARCHAR(length=50),
 'operator':db.types.NVARCHAR(length=70),
 'route':db.types.NVARCHAR(length=70),
 'ac_type':db.types.NVARCHAR(length=50),
 'registration':db.types.NVARCHAR(length=20),
 'all_aboard':db.types.INTEGER,
 'pass_aboard':db.types.INTEGER,
 'crew_aboard':db.types.INTEGER,
 'total_fatalities':db.types.INTEGER,
 'pass_fatalities':db.types.INTEGER,
 'crew_fatalities':db.types.INTEGER,
 'ground':db.types.INTEGER,
 'summary':db.types.NVARCHAR(length=2700)}

df_oaci.to_sql('accidentes', conexion, index=False, dtype=types_oaci, if_exists='replace') #envío de dataframe de accidentes preprocesado a la base de datos

5008

In [144]:
# definición previa de un diccionario con los tipos con los que se desea ingestar la data en la base de datos
# y envío de datos de pasajeros transportados vía aérea a la base de datos
types_pass={
 'country':db.types.NVARCHAR(length=100),
 'anio':db.types.DATE,
 'passengers':db.types.BIGINT}

df_pass.to_sql('passenger', conexion, index=False, dtype=types_pass, if_exists='replace')  #envío de dataframe de pasajeros preprocesado a la base de datos

9723

In [145]:
conexion.close() # cierre de la conexión con la base de datos